In [1]:
import glob
import os
import fuse, causal_utils
glob.glob("/ist/ist-share/scads/can/robust_nlu/korn_reweight/nli/*/")

['/ist/ist-share/scads/can/robust_nlu/korn_reweight/nli/outputs_bert_base_overlapping_korn_ps3class_3/',
 '/ist/ist-share/scads/can/robust_nlu/korn_reweight/nli/outputs_bert_base_overlapping_korn_ps3class_2/',
 '/ist/ist-share/scads/can/robust_nlu/korn_reweight/nli/outputs_bert_base_overlapping_korn_ps3class_1/']

In [2]:
!ls /raid/can/nli_models/poe3

nli  outputs_poe_bert_base_korn_clark_1_seed33370


In [3]:
import pandas as pd
import numpy as np
from scipy.special import softmax
from scipy.special import expit

# select model
model_path='/raid/can/nli_models/poe3/nli/outputs_poe_bert_base_korn_clark_1_seed13370/'
# select data path
data_path='/ist/users/canu/debias_nlu/data/nli/'
df = pd.read_json(model_path+'raw_train.jsonl', lines=True)
# select fusion method here
fusion = fuse.sum_fuse

#  get score from the bias model

In [38]:
# avg prob
#s score from bias model
df_hans = pd.read_json(data_path+'hans_prob_korn_lr_overlapping_sample_weight_3class.jsonl', lines=True)
hans_score=[b for b in df_hans['bias_probs'] ]
hans_score=np.array(hans_score)

# Get avg  prob of bert

In [39]:
list_probs = []
for i in df['probs']:
    list_probs.extend(i)
x=np.array(list_probs)
avg=np.average(x,axis=0)

# y1m0

In [40]:
bias_score=fusion(avg,hans_score)
#y1m0

In [41]:
!ls /raid/can/nli_models/poe3/outputs_poe_bert_base_korn_clark_1_seed13370/

ls: cannot access '/raid/can/nli_models/poe3/outputs_poe_bert_base_korn_clark_1_seed13370/': No such file or directory


In [42]:
result_path=model_path+'normal/'
df_bert = pd.read_json(result_path+'hans_result.jsonl', lines=True)

In [43]:
# ent = []
y1m1prob = []
for p,h in zip(df_bert['probs'],hans_score):
    new_y1m1 = fusion(np.array(p),h)
    y1m1prob.append(new_y1m1)
  

# corrected y0m0

In [10]:
c = sharpness_correction(hans_score, y1m1prob) 
bias_score = fusion(c,hans_score)

NameError: name 'sharpness_correction' is not defined

# TIE

In [44]:
debias_scores = []
for p,b in zip(y1m1prob,bias_score):
    debias_scores.append(p-b) # TIE  

# Eval HANS

In [45]:
key = {0:"entailment",1:"contradiction",2:"neutral"}
labels = []
for i in debias_scores:
    labels.append(key[np.argmax(i)])
df_bert['debias_label']=labels

In [46]:
text_ans=""
for idx, obj in enumerate(df_bert['label']):
    text_ans = text_ans + "ex"+str(idx)+","+obj+"\n"   
    
text_ans_debias=""
for idx, obj in enumerate(df_bert['debias_label']):
    text_ans_debias = text_ans_debias + "ex"+str(idx)+","+obj+"\n"          

In [47]:
def format_label(label):
    if label == "entailment":
        return "entailment"
    else:
        return "non-entailment"

guess_dict = {}
for line in text_ans.split("\n"):
    if len(line)>1:
        parts = line.strip().split(",")
        guess_dict[parts[0]] = format_label(parts[1])
        
guess_dict_debias = {}
for line in text_ans_debias.split("\n"):
    if len(line)>1:
        parts = line.strip().split(",")
        guess_dict_debias[parts[0]] = format_label(parts[1])        

In [48]:
labels,baseline_avg=causal_utils.get_heur(guess_dict) # normal

Heuristic entailed results:
lexical_overlap: 0.9734
subsequence: 0.9968
constituent: 0.9924

Heuristic non-entailed results:
lexical_overlap: 0.7482
subsequence: 0.1114
constituent: 0.3026
avg: 0.6874666666666668


In [49]:
labels,debias_avg=causal_utils.get_heur(guess_dict_debias) # TIE

Heuristic entailed results:
lexical_overlap: 0.9724
subsequence: 0.9966
constituent: 0.9912

Heuristic non-entailed results:
lexical_overlap: 0.7528
subsequence: 0.116
constituent: 0.3132
avg: 0.6903666666666667


# Causal Mediation Analysis

In [50]:
def get_ans(ans):
    if ans == 0:
        return 'entailment'
    else:
        return 'non-entailment' 
    
# y0m0
import pickle
modelname='mnli_lr_model.sav'
loaded_model = pickle.load(open(modelname, 'rb'))
x0=loaded_model.predict_proba(np.array([[0,0,0.41997876976119086]]))
m0=avg
y0m0=fusion(x0,m0)    

In [51]:
factual_pred_correct = []
TIE_pred_correct = []
NIE_pred_correct = []
INTmed_pred_correct = []
pred_correct = []
all_TIE = []
all_NIE = []
all_NDE = []
all_INTmed = []
for i in range(len(labels)): 
    y1m1 = y1m1prob[i]
    y1m0 = bias_score[i]
    TE = y1m1 - y0m0
    NDE = bias_score[i] - y0m0
    y0m1= fusion(x0,np.array(df_bert['probs'][i]) )
    TIE = y1m1 - y1m0
    NIE = y0m1 - y0m0
    INTmed = TIE - NIE
    # factual
    factual_ans = np.argmax(df_bert['probs'][i])
    factual_ans = get_ans(factual_ans)
    factual_correct = factual_ans==labels[i]   
    factual_pred_correct.append(factual_correct)
    # TIE
    TIE_ans = np.argmax(TIE)
    TIE_ans = get_ans(TIE_ans)
    TIE_correct = TIE_ans==labels[i]  
    TIE_pred_correct.append(TIE_correct)
    # INTmed
    INTmed_ans = np.argmax(INTmed[0])
    INTmed_ans = get_ans(INTmed_ans)
    INTmed_correct = INTmed_ans==labels[i]  
    INTmed_pred_correct.append(INTmed_correct)
    # NIE
    NIE_ans = np.argmax(NIE[0])
    NIE_ans = get_ans(NIE_ans)
    NIE_correct = NIE_ans==labels[i]  
    NIE_pred_correct.append(NIE_correct)    
    
    # save
    all_NDE.append(NDE[0][0])
    all_NIE.append(NIE[0][0])
    all_TIE.append(TIE[0])
    all_INTmed.append((INTmed[0][0]))
    if  (TIE[0]/TE[0][0])<9999999:
        cf_ans = np.argmax(np.array(df_bert['probs'][i]-hans_score[i]))
        cf_ans = get_ans(cf_ans)  
        cf_correct = cf_ans==labels[i]
    else:
#         print(cf_ans)
        cf_correct = factual_ans ==labels[i]
    pred_correct.append(cf_correct)
    
#     np.array(df_bert['probs'][i]-bias_score)
#     labels[i]
print(np.array(all_TIE).mean(),(np.array(all_TIE).std()))
print(np.array(all_INTmed).mean(),(np.array(all_INTmed).std()))
print(np.array(all_NIE).mean(),(np.array(all_NIE).std()))

0.08649503037239671 0.07753374427816435
-0.03193751895445736 0.026914285032747287
0.11843254932685407 0.10443938146612336


In [52]:
 np.argmax(NIE)

0

In [53]:
NIE

array([[ 0.18147684, -0.12462973, -0.12555245]])

In [54]:
print(sum(NIE_pred_correct)/30000,sum(INTmed_pred_correct)/30000)

0.6872666666666667 0.3651


In [55]:
print(sum(factual_pred_correct)/30000,sum(pred_correct)/30000,sum(TIE_pred_correct)/30000)

0.6874666666666667 0.7312 0.6903666666666667


In [34]:
#TIE # 0.032416620996870094 # utama best seed  0.04260506151060745
#TIE # 0.0879242620437139 # PoE
#TIE # 0.10864416858494 # Baseline 0.10295758381662952
#
# baseline
# reweight_korn
# poe_korn


In [33]:
!ls /raid/can/nli_models/

baseline  check_baseline  korn_reweight  poe  poe2  poe3  reweight_utama_github


In [20]:
!python cma.py

Heuristic entailed results:
lexical_overlap: 0.9806
subsequence: 0.9874
constituent: 0.9958

Heuristic non-entailed results:
lexical_overlap: 0.7372
subsequence: 0.1222
constituent: 0.1922
avg: 0.6692333333333332
0.09262385912553528 0.07259367040949645
-0.03402964607050396 0.02528866896020864
0.12665350519603927 0.09787408550005666
Heuristic entailed results:
lexical_overlap: 0.9896
subsequence: 0.9994
constituent: 0.9998

Heuristic non-entailed results:
lexical_overlap: 0.6146
subsequence: 0.056
constituent: 0.0526
avg: 0.6186666666666666
0.10295758381662952 0.06343940052637884
-0.03768225306967608 0.022113234206897377
0.1406398368863056 0.08554443506460485
Heuristic entailed results:
lexical_overlap: 0.9408
subsequence: 0.9904
constituent: 0.9978

Heuristic non-entailed results:
lexical_overlap: 0.7556
subsequence: 0.1878
constituent: 0.2064
avg: 0.6798000000000002
0.08612845423015646 0.07794352316021892
-0.031777019036895594 0.027034823388913467
0.11790547326705204 0.104969423443100

In [23]:
!python cma.py

Heuristic entailed results:
lexical_overlap: 0.9838
subsequence: 0.9946
constituent: 0.9978

Heuristic non-entailed results:
lexical_overlap: 0.7264
subsequence: 0.1144
constituent: 0.2728
avg: 0.6816333333333334
0.09049318935295611 0.07330273608231962
-0.033259518297814955 0.025551918800548676
0.12375270765077105 0.09884589944548612
Heuristic entailed results:
lexical_overlap: 0.9784
subsequence: 0.9986
constituent: 0.9998

Heuristic non-entailed results:
lexical_overlap: 0.6494
subsequence: 0.0844
constituent: 0.1872
avg: 0.6496333333333333
0.09638890603753264 0.06965581836013897
-0.035384280694031746 0.024282320036851667
0.13177318673156438 0.09392968074471238
Heuristic entailed results:
lexical_overlap: 0.9756
subsequence: 0.9966
constituent: 0.9952

Heuristic non-entailed results:
lexical_overlap: 0.7424
subsequence: 0.1658
constituent: 0.285
avg: 0.6934333333333332
0.0880008136703304 0.07883599964955158
-0.03250504798919367 0.027354145127177664
0.1205058616595241 0.10618183988554

In [32]:
!python cma.py

Heuristic entailed results:
lexical_overlap: 0.9456
subsequence: 0.9914
constituent: 0.9972

Heuristic non-entailed results:
lexical_overlap: 0.8306
subsequence: 0.1218
constituent: 0.406
avg: 0.7154333333333333
0.08000372094798096 0.0787561937491391
-0.02949995560149599 0.027361459893534817
0.10950367654947694 0.10610774652291177
Heuristic entailed results:
lexical_overlap: 0.9574
subsequence: 0.9874
constituent: 0.9918

Heuristic non-entailed results:
lexical_overlap: 0.6906
subsequence: 0.0794
constituent: 0.3716
avg: 0.6797000000000001
0.0879242620437139 0.07298768722612486
-0.03225202938627739 0.02546727282854149
0.12017629142999127 0.09844616676694962
Heuristic entailed results:
lexical_overlap: 0.9734
subsequence: 0.9968
constituent: 0.9924

Heuristic non-entailed results:
lexical_overlap: 0.7482
subsequence: 0.1114
constituent: 0.3026
avg: 0.6874666666666668
0.08649503037239671 0.07753374427816435
-0.03193751895445736 0.026914285032747287
0.11843254932685407 0.10443938146612336

In [56]:
!python cma.py

Heuristic entailed results:
lexical_overlap: 0.6114
subsequence: 0.7946
constituent: 0.8006

Heuristic non-entailed results:
lexical_overlap: 0.8062
subsequence: 0.4662
constituent: 0.6038
avg: 0.6804666666666667
0.03643628258946173 0.08115885570442263
-0.013843901881157371 0.027830165672163277
0.0502801844706191 0.10897557626360897
Heuristic entailed results:
lexical_overlap: 0.6072
subsequence: 0.7978
constituent: 0.8328

Heuristic non-entailed results:
lexical_overlap: 0.8622
subsequence: 0.4736
constituent: 0.5834
avg: 0.6928333333333333
0.03180075964343514 0.07565075366276272
-0.012029739993108565 0.025821467010905725
0.04383049963654371 0.10145741044910482
Heuristic entailed results:
lexical_overlap: 0.7112
subsequence: 0.8808
constituent: 0.896

Heuristic non-entailed results:
lexical_overlap: 0.873
subsequence: 0.4076
constituent: 0.5794
avg: 0.7246666666666667
0.04260506151060745 0.08500409682526075
-0.0161633969318506 0.029117621071396035
0.058768458442458045 0.11410886837115

In [6]:
import cma
import glob
import numpy as np
import os
import fuse, causal_utils
model_path='/raid/can/nli_models/korn_reweight/'
task='nli'
data_path='/ist/users/canu/debias_nlu/data/' + task + '/'
fusion = fuse.sum_fuse
test_set='test'

In [7]:
cma.report_CMA(model_path,task,data_path,test_set,fusion)

-0.02543364955731894 0.13072994760352968
-0.004192283378247885 0.013030681614384764
-0.021241366179071058 0.1346177304922089
-0.0250747032937291 0.13164532145365193
-0.004196849611183269 0.013124232712194879
-0.020877853682545838 0.13553195277065558
-0.02503439801379185 0.13140930408871812
-0.0041369468423411255 0.01312457364288504
-0.020897451171450726 0.1352656790889345
factual score:
[0.8466232709519935, 0.8489625711960944, 0.8494711147274207] 0.8483523189585029 0.001240123485094363
TE:
[-0.020517851995059046, -0.020167172431678024, -0.020112573340046538] -0.020265865922261202 0.00017956985802152476
TIE:
[-0.02543364955731894, -0.0250747032937291, -0.02503439801379185] -0.02518091695494663 0.00017946486215805643
[0.846826688364524, 0.8486574450772986, 0.8494711147274207] 0.8483184160564144 0.001105879199233784
NIE:
[-0.021241366179071058, -0.020877853682545838, -0.020897451171450726] -0.02100555701102254 0.00016693409516861175
[0.8466232709519935, 0.8488608624898292, 0.8489625711960

In [20]:
  df_bert = pd.read_json('/raid/can/nli_models/poe3/nli/outputs_poe_bert_base_korn_clark_1_seed13370/'+'raw_mm.jsonl', lines=True)

NameError: name 'pd' is not defined

In [5]:
!ls /raid/can/nli_models/

baseline  check_baseline  korn_reweight  poe  poe2  poe3  reweight_utama_github


In [7]:
!ls -ltr /raid/can/nli_models/baseline/nli/outputs_bert_base_clark_1_seed13370/normal/


total 2406868
-rw-rw-r-- 1 canu canu        66 Nov 30 13:43 spelling_result.txt
-rw-rw-r-- 1 canu canu        66 Nov 30 13:43 negation_result.txt
-rw-rw-r-- 1 canu canu        66 Nov 30 13:43 numerical_reasoning_result.txt
-rw-rw-r-- 1 canu canu        66 Nov 30 13:43 length_mismatch_result.txt
-rw-rw-r-- 1 canu canu        65 Nov 30 13:43 antonym_result.txt
-rw-rw-r-- 1 canu canu        66 Nov 30 13:43 word_overlap_result.txt
-rw-rw-r-- 1 canu canu 193014347 Dec  1 00:08 distill_output_1.jsonl
drwxrwxr-x 3 canu canu      4096 Dec  1 01:07 distill_model_1
-rw-rw-r-- 1 canu canu 615717404 Dec  1 05:01 raw_train_result_2.jsonl
drwxrwxr-x 4 canu canu      4096 Dec  2 00:10 distill_model_0
-rw-rw-r-- 1 canu canu 193128135 Dec  2 04:52 distill_output_0.jsonl
-rw-rw-r-- 1 canu canu        67 Dec  7 22:36 result.txt
-rw-rw-r-- 1 canu canu        66 Dec  7 22:36 snli_hard_result.txt
-rw-rw-r-- 1 canu canu        66 Dec  7 22:36 mnli_hard_dev_mm_result.txt
-rw-rw-r-- 1 canu canu        52 Dec  

In [22]:
import pandas as pd

In [23]:
  df_bias_model = pd.read_json(data_path+'test'+'_prob_korn_lr_overlapping_sample_weight_3class.jsonl', lines=True)

In [34]:
if '-' in df_bias_model['gold_label'].value_counts():
    print(df_bias_model['gold_label'].value_counts()['-'])

168


In [9]:
df_bias_model

NameError: name 'df_bias_model' is not defined